## Making DataWarehouse


In [177]:
import datetime as dt
import urllib.request
from IPython.display import Image
from IPython.core.display import HTML
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import re

##### Importing the data

In [178]:
def generate_sample(filename, s):
    n = 2974335
    indices = np.arange(1, n + 1)
    skip = np.sort(np.random.choice(indices, size=n - s, replace=False))
    return pd.read_csv(filename, skiprows=skip)

In [179]:
us_acidents = pd.read_csv('https://www.dropbox.com/s/i4bnh4ynkoka0po/Accidents.csv?dl=1')

us_acidents.head(5)

us_acidents = us_acidents[us_acidents["State"] != 'DC'].reset_index(drop=True)

C:\Users\darsh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


##### Importing the event data

In [180]:
event_code = pd.read_csv("https://docs.google.com/uc?export=download&id=1gcxQcC65sIGEjQSJ1CfPfFWAlbeXoNHf", delimiter=';')
event_code = event_code.rename(columns={"Code": "TMC_Key", "Description":"Event"})
event_code = event_code.append({'TMC_Key': -1, 'Event': 'unidentified'}, ignore_index=True)
event_code.head(5)

,TMC_Key,Event
0,1,traffic problem
1,2,queuing traffic (with average speeds Q). Dange...
2,11,overheight warning system triggered
3,12,"(Q) accident(s), traffic being directed around..."
4,16,"closed, rescue and recovery work in progress"


##### Importing Alcohol Consumption Data set

In [181]:
def discretize_rate(x):
    if x <= 2.0:
        return '(Low) rate <= 2.0'
    elif 2.0 < x <= 2.5:
        return '(Medium) 2.0 < rate <= 2.5'
    elif 2.5 < x <= 3.5:
        return '(High) 2.5 < rate <= 3.5'
    else:
        return '(Very High) rate > 3.5'

alcohol_consumption = pd.read_csv("https://docs.google.com/uc?export=download&id=1G9Mk-VtCNFo8wHf1qWDdgrEj5KmAhemY", delimiter=';')

alcohol_consumption.at[20, "State"] = "Massachusetts"

alcohol_consumption = alcohol_consumption.sort_values(by=['rate'], ascending=False).reset_index(drop=True)

alcohol_consumption['DiscreteRate'] = alcohol_consumption['rate'].map(discretize_rate)

alcohol_dict = pd.Series(alcohol_consumption['DiscreteRate'].values, index=alcohol_consumption['State']).to_dict()
alcohol_dict

{'New Hampshire': '(Very High) rate > 3.5',
 'Delaware': '(Very High) rate > 3.5',
 'Nevada': '(High) 2.5 < rate <= 3.5',
 'North Dakota': '(High) 2.5 < rate <= 3.5',
 'Montana': '(High) 2.5 < rate <= 3.5',
 'Vermont': '(High) 2.5 < rate <= 3.5',
 'Wisconsin': '(High) 2.5 < rate <= 3.5',
 'Idaho': '(High) 2.5 < rate <= 3.5',
 'South Dakota': '(High) 2.5 < rate <= 3.5',
 'Colorado': '(High) 2.5 < rate <= 3.5',
 'Maine': '(High) 2.5 < rate <= 3.5',
 'Alaska': '(High) 2.5 < rate <= 3.5',
 'Minnesota': '(High) 2.5 < rate <= 3.5',
 'Oregon': '(High) 2.5 < rate <= 3.5',
 'Wyoming': '(High) 2.5 < rate <= 3.5',
 'Hawaii': '(High) 2.5 < rate <= 3.5',
 'Florida': '(High) 2.5 < rate <= 3.5',
 'Rhode Island': '(High) 2.5 < rate <= 3.5',
 'Massachusetts': '(High) 2.5 < rate <= 3.5',
 'Louisiana': '(High) 2.5 < rate <= 3.5',
 'Missouri': '(High) 2.5 < rate <= 3.5',
 'Connecticut': '(Medium) 2.0 < rate <= 2.5',
 'Iowa': '(Medium) 2.0 < rate <= 2.5',
 'Illinois': '(Medium) 2.0 < rate <= 2.5',
 'Texas'

In [182]:
def discretize_total(x):
    if x <= 5000000:
        return '(Low) registrations <= 5000000'
    elif 5000000 < x <= 10000000:
        return '(Medium) 5000000 < registrations <= 10000000'
    elif 10000000 < x <= 20000000:
        return '(High) 10000000 < registrations <= 20000000'
    else:
        return '(Very High) registrations > 20000000'

vehicle_registration = pd.read_csv("https://docs.google.com/uc?export=download&id=11fq6PjFn7RdjvGyTaEHvqwkKwW_zdmWe", delimiter=';')
vehicle_registration = vehicle_registration[vehicle_registration['State'] != 'Dist. of Col.']

print(vehicle_registration)

vehicle_registration['Total'] = vehicle_registration['Total'].map(lambda x: int(re.sub(' ', '', x)))

vehicle_registration = vehicle_registration.sort_values(by=['Total'], ascending=False).reset_index(drop=True)

vehicle_registration['DiscreteTotal'] = vehicle_registration['Total'].map(discretize_total)

vehicle_dict = pd.Series(vehicle_registration['DiscreteTotal'].values, index=vehicle_registration['State']).to_dict()
vehicle_dict

             State        Total
0          Alabama   5 300 199 
1           Alaska     803 684 
2          Arizona   5 806 313 
3         Arkansas   2 817 145 
4       California  31 022 328 
5         Colorado   5 356 018 
6      Connecticut   2 879 802 
7         Delaware     1008468 
9          Florida    17496002 
10         Georgia     8512550 
11          Hawaii     1267385 
12           Idaho     1879670 
13        Illinois    10588910 
14         Indiana     6190736 
15            Iowa     3691892 
16          Kansas     2684010 
17        Kentucky     4368285 
18       Louisiana     3885119 
19           Maine     1125588 
20        Maryland     4204846 
21   Massachusetts     5061499 
22        Michigan     8386831 
23       Minnesota     5404277 
24     Mississippi     2067498 
25        Missouri     5498675 
26         Montana     1845338 
27        Nebraska     1961309 
28          Nevada     2514338 
29   New Hampshire     1346318 
30      New Jersey     6055389 
31      

{'California': '(Very High) registrations > 20000000',
 'Texas': '(Very High) registrations > 20000000',
 'Florida': '(High) 10000000 < registrations <= 20000000',
 'New York': '(High) 10000000 < registrations <= 20000000',
 'Ohio': '(High) 10000000 < registrations <= 20000000',
 'Pennsylvania': '(High) 10000000 < registrations <= 20000000',
 'Illinois': '(High) 10000000 < registrations <= 20000000',
 'Georgia': '(Medium) 5000000 < registrations <= 10000000',
 'Michigan': '(Medium) 5000000 < registrations <= 10000000',
 'North Carolina': '(Medium) 5000000 < registrations <= 10000000',
 'Virginia': '(Medium) 5000000 < registrations <= 10000000',
 'Washington': '(Medium) 5000000 < registrations <= 10000000',
 'Indiana': '(Medium) 5000000 < registrations <= 10000000',
 'New Jersey': '(Medium) 5000000 < registrations <= 10000000',
 'Arizona': '(Medium) 5000000 < registrations <= 10000000',
 'Tennessee': '(Medium) 5000000 < registrations <= 10000000',
 'Wisconsin': '(Medium) 5000000 < regis

##### Importing information about areas 

In [183]:
urban = pd.read_excel('https://docs.google.com/uc?export=download&id=1Ccn8-oZZ5HLjwZdK3gWIdVcnCswBYm_l')

urban = urban.drop(['State Abr.','CBSA title', 'CBSA 2012 pop', 'County 2012 pop'], axis=1)

urban['County name'] = urban['County name'].map(lambda x: re.sub(' County', '', x))

urban_term = {1:'Large Central Metro',
              2:'Large Fringe Metro',
              3:'Medium Metro',
              4:'Small Metro',
              5:'Micropolitan',
              6:'Non-Core'}

urban["2013 code"] = urban["2013 code"].map(urban_term)

urban_dict = pd.Series(urban['2013 code'].values, index=urban['County name']).to_dict()
urban_dict

{'Autauga': 'Medium Metro',
 'Baldwin': 'Micropolitan',
 'Barbour': 'Non-Core',
 'Bibb': 'Small Metro',
 'Blount': 'Medium Metro',
 'Bullock': 'Non-Core',
 'Butler': 'Large Fringe Metro',
 'Calhoun': 'Non-Core',
 'Chambers': 'Large Fringe Metro',
 'Cherokee': 'Micropolitan',
 'Chilton': 'Large Fringe Metro',
 'Choctaw': 'Non-Core',
 'Clarke': 'Large Fringe Metro',
 'Clay': 'Small Metro',
 'Cleburne': 'Non-Core',
 'Coffee': 'Micropolitan',
 'Colbert': 'Small Metro',
 'Conecuh': 'Non-Core',
 'Coosa': 'Micropolitan',
 'Covington': 'Non-Core',
 'Crenshaw': 'Non-Core',
 'Cullman': 'Micropolitan',
 'Dale': 'Micropolitan',
 'Dallas': 'Large Central Metro',
 'DeKalb': 'Non-Core',
 'Elmore': 'Micropolitan',
 'Escambia': 'Medium Metro',
 'Etowah': 'Small Metro',
 'Fayette': 'Small Metro',
 'Franklin': 'Medium Metro',
 'Geneva': 'Small Metro',
 'Greene': 'Small Metro',
 'Hale': 'Micropolitan',
 'Henry': 'Micropolitan',
 'Houston': 'Non-Core',
 'Jackson': 'Non-Core',
 'Jefferson': 'Micropolitan',


##### Importing speed limits of US states

In [184]:
def discretize_speed(x):
    if x < 70:
        return '(Low) limit < 70 '
    elif 70 <= x < 80:
        return '(Medium) 70 <= limit < 80'
    else:
        return '(High) limit > 80'

speed_limits = pd.read_excel('https://docs.google.com/uc?export=download&id=1WPxr8k9w5fGOgL687BtCvlCHQYF0i1zV')

speed_limits = speed_limits.drop(['Freeway (trucks)','Freeway (urban)','Divided (rural)','Undivided (rural)','Residential'], axis=1)

speed_limits = speed_limits.rename(columns={'State or territory': 'State', 'Freeway (rural)': 'Max Speed Limit (mph)'})

us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
             'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
             'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
             'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
             'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
             'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
             'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas','Utah',
             'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

speed_limits['State'] = speed_limits['State'].map(lambda x: re.sub('[^a-z A-Z]+', '', x), na_action='ignore')
speed_limits['Max Speed Limit (mph)'] = speed_limits['Max Speed Limit (mph)'].map(lambda x: re.sub('\(.*', '', x), na_action='ignore')

speed_limits = speed_limits[speed_limits['State'].isin(us_states)]

speed_limits['Max Speed Limit (mph)'] = speed_limits['Max Speed Limit (mph)'].map(lambda x: int(x[:-4][-3:]))
speed_limits.at[41, 'Max Speed Limit (mph)'] = 70

speed_limits = speed_limits.sort_values(by=['Max Speed Limit (mph)'], ascending=False).reset_index(drop=True)

speed_limits['Discrete Max Speed Limit'] = speed_limits['Max Speed Limit (mph)'].map(discretize_speed)

speed_dict = pd.Series(speed_limits['Discrete Max Speed Limit'].values, index=speed_limits['State']).to_dict()
speed_dict

{'Texas': '(High) limit > 80',
 'Montana': '(High) limit > 80',
 'Idaho': '(High) limit > 80',
 'Utah': '(High) limit > 80',
 'South Dakota': '(High) limit > 80',
 'Nevada': '(High) limit > 80',
 'Wyoming': '(High) limit > 80',
 'Maine': '(Medium) 70 <= limit < 80',
 'Michigan': '(Medium) 70 <= limit < 80',
 'North Dakota': '(Medium) 70 <= limit < 80',
 'Colorado': '(Medium) 70 <= limit < 80',
 'Nebraska': '(Medium) 70 <= limit < 80',
 'Arizona': '(Medium) 70 <= limit < 80',
 'Arkansas': '(Medium) 70 <= limit < 80',
 'Kansas': '(Medium) 70 <= limit < 80',
 'Oklahoma': '(Medium) 70 <= limit < 80',
 'Louisiana': '(Medium) 70 <= limit < 80',
 'New Mexico': '(Medium) 70 <= limit < 80',
 'Ohio': '(Medium) 70 <= limit < 80',
 'Oregon': '(Medium) 70 <= limit < 80',
 'Pennsylvania': '(Medium) 70 <= limit < 80',
 'New Hampshire': '(Medium) 70 <= limit < 80',
 'South Carolina': '(Medium) 70 <= limit < 80',
 'California': '(Medium) 70 <= limit < 80',
 'Tennessee': '(Medium) 70 <= limit < 80',
 'V

#### Transforming the data
 

In [185]:
us_acidents.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [186]:
us_acidents = us_acidents.drop(['Source', 'Start_Lat', 'Country', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'Number', 'Side', 'Zipcode', 'Airport_Code', 'Weather_Timestamp', 'Pressure(in)', 'Wind_Chill(F)'], axis=1)

In [187]:
us_acidents = us_acidents.dropna(subset=['City', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']).reset_index(drop=True)

In [188]:
us_acidents['TMC'].fillna(-1, inplace=True)
us_acidents['TMC'] = us_acidents['TMC'].astype(int)

In [189]:
timezone_df = us_acidents[['State', 'Timezone']].drop_duplicates(subset="State")
timezone_dict = pd.Series(timezone_df['Timezone'].values, index=timezone_df['State']).to_dict()

us_acidents['Timezone'].fillna(us_acidents['State'].map(timezone_dict), inplace=True)

### Location Dimension

In [190]:
us_acidents.insert(us_acidents.columns.get_loc('Street'), 'LocationKey', range(1, len(us_acidents) + 1))

In [191]:
us_acidents['State'].unique()

array(['OH', 'WV', 'CA', 'FL', 'GA', 'SC', 'NE', 'IA', 'IL', 'MO', 'WI',
       'IN', 'MI', 'NJ', 'NY', 'CT', 'MA', 'RI', 'NH', 'PA', 'KY', 'MD',
       'VA', 'DE', 'TX', 'WA', 'OR', 'AL'], dtype=object)

In [192]:
replace_values = {'OH':'Ohio', 'WV':'West Virginia', 'CA': 'California', 'FL': 'Florida', 'GA': 'Georgia', 'SC':'South Carolina', 'NE': 'Nebraska', 'IA': 'Iowa', 'IL': 'Illinois', 'MO': 'Missouri', 'WI': 'Wisconsin',
       'IN': 'Indiana', 'MI': 'Michigan', 'NJ': 'New Jersey', 'NY': 'New York', 'CT': 'Connecticut', 'MA': 'Massachusetts', 'RI': 'Rhode Island', 'NH': 'New Hampshire', 'PA': 'Pennsylvania', 'KY': 'Kentucky', 'MD': 'Maryland',
       'VA': 'Virginia', 'DE': 'Delaware', 'TX':'Texas', 'WA': 'Washington', 'OR': 'Oregon', 'AL': 'Alabama', 'NC': 'North Carolina', 'MN': 'Minnesota', 'OK': 'Oklahoma', 'LA': 'Louisiana',
   'TN': 'Tennessee', 'UT': 'Utah', 'CO': 'Colorado', 'AZ': 'Arizona', 'NV': 'Nevada', 'KS': 'Kansas', 'MS': 'Mississippi', 'NM': 'New Mexico', 'ME': 'Maine', 'AR': 'Arkansas', 'WY': 'Wyoming','VT': 'Vermont', 'ID': 'Idaho', 'ND': 'North Dakota', 'MT': 'Montana', 'SD': 'South Dakota'}

In [193]:
us_acidents = us_acidents.replace({"State": replace_values})

In [194]:
sub_regions = {
    'New England': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'],
    'Mid Atlantic': ['New Jersey', 'New York','Pennsylvania'],
    'East North Central': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin'],
    'West North Central': ['Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota'],
    'South Atlantic': ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','West Virginia'],
    'East South Central': ['Alabama','Kentucky','Mississippi','Tennessee'],
    'West South Central': ['Arkansas','Louisiana','Oklahoma','Texas'],
    'Mountain': ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah','Wyoming'],
    'Pacific': ['Alaska','California','Hawaii','Oregon','Washington']
}

regions = {
    'Northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York','Pennsylvania'],
    'Midwest': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota'],
    'South': ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','District of Columbia','West Virginia', 'Alabama','Kentucky','Mississippi','Tennessee', 'Arkansas','Louisiana','Oklahoma','Texas'],
    'West': ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah','Wyoming', 'Alaska','California','Hawaii','Oregon','Washington']
}

regions_dict = dict([(v, k) for k, l in regions.items() for v in l])
sub_regions_dict = dict([(v, k) for k, l in sub_regions.items() for v in l])

##### Inserting region attributes

In [195]:
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'Subregion', us_acidents['State'].map(sub_regions_dict))
us_acidents['Subregion'].value_counts()

Pacific               186641
South Atlantic         92003
West South Central     80490
Mid Atlantic           67134
East North Central     56266
New England            14636
West North Central     12903
East South Central        10
Name: Subregion, dtype: int64

In [196]:
us_acidents.insert(us_acidents.columns.get_loc("Timezone"), 'Region', us_acidents['State'].map(regions_dict))
us_acidents['Region'].value_counts()

West         186641
South        172503
Northeast     81770
Midwest       69169
Name: Region, dtype: int64

##### Adding alcohol consumption rate

In [197]:
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'AlcoholConsumptionRate', us_acidents['State'].map(alcohol_dict))
us_acidents['AlcoholConsumptionRate'].value_counts()

(Medium) 2.0 < rate <= 2.5    428979
(High) 2.5 < rate <= 3.5       61672
(Low) rate <= 2.0              19139
(Very High) rate > 3.5           293
Name: AlcoholConsumptionRate, dtype: int64

#### Inserting registering information

In [198]:
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'NumberOfVehicleRegistrations', us_acidents['State'].map(vehicle_dict))
us_acidents['NumberOfVehicleRegistrations'].value_counts()

(Very High) registrations > 20000000            255660
(High) 10000000 < registrations <= 20000000     137292
(Medium) 5000000 < registrations <= 10000000     84770
(Low) registrations <= 5000000                   32361
Name: NumberOfVehicleRegistrations, dtype: int64

#### Classifing Urban and Rural accidents 

In [199]:
# add rate
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'UrbanRuralClassification', us_acidents['County'].map(urban_dict))

us_acidents = us_acidents[us_acidents['UrbanRuralClassification'].notna()]
us_acidents.reset_index(drop=True, inplace=True)

us_acidents['UrbanRuralClassification'].value_counts()

Large Central Metro    236294
Large Fringe Metro      83112
Non-Core                70634
Medium Metro            58089
Small Metro             29031
Micropolitan            19683
Name: UrbanRuralClassification, dtype: int64

#### Inserting max speed related information

In [200]:
us_acidents.insert(us_acidents.columns.get_loc('Timezone'), 'MaximumSpeedLimit', us_acidents['State'].map(speed_dict))

us_acidents['MaximumSpeedLimit'].value_counts()

(Medium) 70 <= limit < 80    366463
(High) limit > 80             80471
(Low) limit < 70              49909
Name: MaximumSpeedLimit, dtype: int64

In [201]:
# create location dataframe
location_dimension = us_acidents[['LocationKey', 'Street','City','County','State','Subregion','Region','AlcoholConsumptionRate','NumberOfVehicleRegistrations','UrbanRuralClassification','MaximumSpeedLimit']]
location_dimension.columns

Index(['LocationKey', 'Street', 'City', 'County', 'State', 'Subregion',
       'Region', 'AlcoholConsumptionRate', 'NumberOfVehicleRegistrations',
       'UrbanRuralClassification', 'MaximumSpeedLimit'],
      dtype='object')

### POI Dimension

In [202]:
us_acidents.insert(us_acidents.columns.get_loc('Amenity'), 'POI_Key', range(1, len(us_acidents) + 1))

In [203]:
poi_dimension = us_acidents.loc[:,'POI_Key':'Turning_Loop']

poi_dimension.columns

Index(['POI_Key', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction',
       'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop'],
      dtype='object')

In [204]:
poi_dimension.head(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,1,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,False,False,False,False,False,False,False,False,False,False,False,True,False
3,4,False,False,False,False,False,False,False,False,False,False,False,False,False
4,5,False,False,False,False,False,False,False,False,False,False,False,True,False


In [205]:
poi_dimension.loc[:,'Amenity':'Turning_Loop'] = poi_dimension.loc[:,'Amenity':'Turning_Loop'].replace({True: 'Present', False: 'Not Present'})

In [206]:
poi_dimension.sample(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
284824,284825,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present
167227,167228,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
277501,277502,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present
489151,489152,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
363564,363565,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present


### Date Dimension

us_acidents['Start_Time'] = pd.to_datetime(us_acidents['Start_Time'])
us_acidents['End_Time'] = pd.to_datetime(us_acidents['End_Time'])
us_acidents = us_acidents.rename(columns={'Start_Time': 'Start_Datetime', 'End_Time': 'End_Datetime'})

In [208]:
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'DateKey', range(1, len(us_acidents) + 1))

##### Inserting start day, start month and start year

In [209]:
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartDay', us_acidents['Start_Datetime'].map(lambda x: x.day))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartMonth', us_acidents['Start_Datetime'].map(lambda x: x.month_name()))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'StartYear', us_acidents['Start_Datetime'].map(lambda x: x.year))

##### Inserting EndDay, EndMonth e EndYear

In [210]:
# add ending date
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndDay', us_acidents['End_Datetime'].map(lambda x: x.day)) 
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndMonth', us_acidents['End_Datetime'].map(lambda x: x.month_name())) 
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'EndYear', us_acidents['End_Datetime'].map(lambda x: x.year))

##### Inserting DayOfWeek

In [211]:
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'DayOfWeek', us_acidents['Start_Datetime'].map(lambda x: x.day_name()))
us_acidents['DayOfWeek'].value_counts()

Thursday     89435
Tuesday      88880
Wednesday    88499
Friday       87249
Monday       83376
Saturday     31527
Sunday       27877
Name: DayOfWeek, dtype: int64

##### Inserting Weekday and Weekend

In [212]:
week_dict = {
    'Monday': 'Week Day',
    'Tuesday': 'Week Day',
    'Wednesday': 'Week Day',
    'Thursday': 'Week Day',
    'Friday': 'Week Day',
    'Saturday': 'Weekend',
    'Sunday': 'Weekend'
}

In [213]:
# add week day/weekend
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WeekdayWeekend', us_acidents['DayOfWeek'].map(week_dict))
us_acidents[['Start_Datetime', 'DayOfWeek', 'WeekdayWeekend']].sample(5)

,Start_Datetime,DayOfWeek,WeekdayWeekend
247966,2016-09-01 10:19:28,Thursday,Week Day
476871,2017-07-06 18:42:33,Thursday,Week Day
262916,2017-01-16 14:50:21,Monday,Week Day
170275,2016-11-15 16:22:51,Tuesday,Week Day
194730,2016-12-30 13:18:09,Friday,Week Day


##### Insereting quarter and week number

In [214]:
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'Quarter', us_acidents['Start_Datetime'].map(lambda x: x.quarter))
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WeekNumber', us_acidents['Start_Datetime'].map(lambda x: x.weekofyear))
us_acidents[['Start_Datetime','Quarter','WeekNumber']].sample(5)

,Start_Datetime,Quarter,WeekNumber
129647,2016-11-28 06:40:18,4,48
472252,2017-06-09 15:03:32,2,23
453546,2017-06-05 12:53:34,2,23
42754,2016-06-20 04:40:40,2,25
128943,2016-11-22 14:58:06,4,47


##### Inserting Holiday Information

In [220]:
federal_holidays = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSRr8UobfETAizkuxMOmzzrxhasDWFWsaWEJ8UJ_-pfq_3g1TnxDDd9NZWth9IJqlrMBjnDeOBBV5Nr/pub?output=csv', delimiter=';')
federal_holidays['date'] = pd.to_datetime(federal_holidays['date']).dt.date
federal_holidays.sample(5)

,date,holiday,"exceptions,,,,,,,,,,,,,,,,"
10,2016-12-26,Christmas Day observed,",,,,,,,,,,,,,,,,"
24,2018-01-15,Martin Luther King Jr. Day,",,,,,,,,,,,,,,,,"
19,2017-10-11,Veterans Day observed,",,,,,,,,,,,,,,,,"
8,2016-11-24,Thanksgiving Day,",,,,,,,,,,,,,,,,"
40,2019-02-09,Labor Day,",,,,,,,,,,,,,,,,"


In [221]:
holidays_dict = pd.Series(federal_holidays['holiday'].values, index=federal_holidays['date']).to_dict()
holidays_dict

{datetime.date(2016, 1, 1): "New Year's Day",
 datetime.date(2016, 1, 18): 'Martin Luther King Jr. Day',
 datetime.date(2016, 2, 15): "Presidents' Day",
 datetime.date(2016, 5, 30): 'Memorial Day',
 datetime.date(2016, 4, 7): 'Independence Day',
 datetime.date(2016, 5, 9): 'Labor Day',
 datetime.date(2016, 10, 10): 'Columbus Day',
 datetime.date(2016, 11, 11): 'Veterans Day',
 datetime.date(2016, 11, 24): 'Thanksgiving Day',
 datetime.date(2016, 12, 25): 'Christmas Day',
 datetime.date(2016, 12, 26): 'Christmas Day observed',
 datetime.date(2017, 1, 1): "New Year's Day",
 datetime.date(2017, 2, 1): "New Year's Day observed",
 datetime.date(2017, 1, 16): 'Martin Luther King Jr. Day',
 datetime.date(2017, 2, 20): "Presidents' Day",
 datetime.date(2017, 5, 29): 'Memorial Day',
 datetime.date(2017, 4, 7): 'Independence Day',
 datetime.date(2017, 4, 9): 'Labor Day',
 datetime.date(2017, 9, 10): 'Columbus Day',
 datetime.date(2017, 10, 11): 'Veterans Day observed',
 datetime.date(2017, 11, 1

In [222]:
# add holiday
us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'Holiday', us_acidents['Start_Datetime'].dt.date.map(holidays_dict))
us_acidents['Holiday'] = us_acidents['Holiday'].fillna('Not Holiday')
us_acidents['Holiday'].value_counts()

Not Holiday                   484969
Independence Day                2238
Veterans Day                    1668
New Year's Day observed         1573
Presidents' Day                 1307
Martin Luther King Jr. Day      1219
Thanksgiving Day                1093
Columbus Day                     847
Labor Day                        818
Christmas Day observed           417
Christmas Day                    326
Memorial Day                     326
New Year's Day                    42
Name: Holiday, dtype: int64

##### Inserting Workday Information

In [223]:
work_days_mask = (us_acidents['Holiday'] != 'Not Holiday') | (us_acidents['WeekdayWeekend'] == 'Weekend')

us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'WorkDay', work_days_mask.map({False: 'Work Day', True: 'Not Work Day'}))
us_acidents['WorkDay'].value_counts()

Work Day        426363
Not Work Day     70480
Name: WorkDay, dtype: int64

In [224]:
us_acidents[us_acidents['WorkDay'] == 'Not Work Day'][['Start_Datetime', 'DayOfWeek', 'Holiday', 'WorkDay']].sample(5)

,Start_Datetime,DayOfWeek,Holiday,WorkDay
268941,2016-11-05 11:59:23,Saturday,Not Holiday,Not Work Day
337221,2017-02-26 13:25:54,Sunday,Not Holiday,Not Work Day
442960,2017-06-10 12:11:49,Saturday,Not Holiday,Not Work Day
75548,2016-11-27 17:54:47,Sunday,Not Holiday,Not Work Day
283308,2016-10-16 07:26:12,Sunday,Not Holiday,Not Work Day


##### Inserting School Holidays

In [227]:
school_holidays = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRkUABljKBQUb6_9YGCB07iMBQElf_l_MpT3Z_ZzhL-W7gpPgYBlMK057443SsuKUtCFJpDnoE6EDaW/pub?gid=1779165823&single=true&output=csv', delimiter=';')
school_holidays['Start']= pd.to_datetime(school_holidays['Start'])
school_holidays['End']= pd.to_datetime(school_holidays['End'])

In [228]:
classes_mask = pd.Series([False for _ in range(len(us_acidents))])

for i in range(len(school_holidays)):
    classes_mask |= (us_acidents['Start_Datetime'] >= school_holidays.loc[i, "Start"]) & (us_acidents['Start_Datetime'] <= school_holidays.loc[i, "End"])

us_acidents.insert(us_acidents.columns.get_loc('Start_Datetime'), 'SchoolBreak', classes_mask.map(lambda x: 'No Classes' if x else 'Classes'))
us_acidents['SchoolBreak'].value_counts()

Classes       288675
No Classes    208168
Name: SchoolBreak, dtype: int64

In [230]:
date_dimension = us_acidents[['DateKey','StartDay','DayOfWeek','WeekdayWeekend','WeekNumber','WorkDay','Holiday','SchoolBreak','StartMonth','Quarter','StartYear','EndDay','EndMonth','EndYear']]

### Weather Dimension


In [231]:
us_acidents.insert(us_acidents.columns.get_loc('Temperature(F)'), 'WeatherKey', range(1, len(us_acidents) + 1))

In [233]:
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)         7092
Humidity(%)            7760
Precipitation(in)    445447
Visibility(mi)         8680
Weather_Condition      8338
Wind_Direction         4198
dtype: int64

In [242]:
def weather_dict_counter(df, column_name):
    dic = defaultdict(Counter)
    for idx in df.index:
        w = df.loc[idx, 'WeekNumber']
        s = df.loc[idx, 'State']
        y = df.loc[idx, 'StartYear']

        v = df.loc[idx, column_name]

        if v is not np.NaN:
            dic[(y, s, w)].update([v])

    return dic

def fill_weather_disc(df, column_name):
    subset = df[['StartYear', 'State', 'WeekNumber', column_name]]
    dic = weather_dict_counter(subset, column_name)
    for idx in df.index:
        if pd.isnull(df.loc[idx, column_name]):
            w = df.loc[idx, 'WeekNumber']
            s = df.loc[idx, 'State']
            y = df.loc[idx, 'StartYear']
            
            try:
                df.loc[idx, column_name] = dic[(y, s, w)].most_common(1)[0][0] # get nested value
            except:
                pass
                

def fill_weather_cont(df, column_name):
    subset = df[['StartYear', 'State', 'WeekNumber', column_name]]
    aggr = subset.groupby(['StartYear', 'State', 'WeekNumber']).mean()
    for idx in df.index:
        if pd.isnull(df.loc[idx, column_name]):
            w = df.loc[idx, 'WeekNumber']
            s = df.loc[idx, 'State']
            y = df.loc[idx, 'StartYear']

            v = aggr.loc[(y, s, w), column_name]

            df.loc[idx, column_name] = v

In [248]:
fill_weather_cont(us_acidents, 'Temperature(F)')


In [249]:
fill_weather_cont(us_acidents, 'Humidity(%)')


In [251]:
fill_weather_cont(us_acidents, 'Precipitation(in)')


In [ ]:
fill_weather_cont(us_acidents, 'Visibility(mi)')


In [252]:
fill_weather_disc(us_acidents, 'Wind_Direction')


In [253]:
fill_weather_disc(us_acidents, 'Weather_Condition')

In [257]:
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)           2
Humidity(%)              2
Precipitation(in)    60437
Visibility(mi)           4
Weather_Condition        3
Wind_Direction           2
dtype: int64

In [258]:
import math

def discretize_humidity(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 25.0:
        return '(Low) rate <= 25.0'
    elif 25.0 < x <= 50.0:
        return '(Medium) 25.0 < rate <= 50.0'
    elif 50.0 < x <= 75.0:
        return '(High) 50.0 < rate <= 75.0'
    else:
        return '(Very High) rate > 75.0'

def discretize_temp(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 32.0:
        return '(Cold) temp <= 32.0'
    elif 32.0 < x <= 50.0:
        return '(Low) 32.0 < temp <= 50.0'
    elif 50.0 < x <= 80.0:
        return '(Moderate) 50.0 < temp <= 80.0'
    elif 80.0 < x <= 100.0:
        return '(Warm) 80.0 < temp <= 100.0'
    else:
        return "(Hot) temp > 100.0"

def discretize_visibility(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 1.0:
        return '(Low) visibility <= 1.0'
    elif 1.0 < x <= 2.0:
        return '(Reduced) 1.0 < visibility <= 2.0'
    elif 2.0 < x <= 5.0:
        return '(Moderate) 2.0 < visibility <= 5.0'
    elif 5.0 < x <= 20.0:
        return '(Good) 5.0 < visibility <= 20.0'
    elif 20.0 < x <= 50.0:
        return '(Very Good) 20.0 < visibility <= 50.0'
    else:
        return '(Excellent) visibility > 50.0'

def discretize_windspeed(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 1.12:
        return '(Calm) speed <= 1.0'
    elif 1.12 < x <= 3.36:
        return '(Light Air) 1.12 < speed <= 3.36'
    elif 3.36 < x <= 7.38:
        return '(Light Breeze) 3.36 < speed <= 7.38'
    elif 7.38 < x <= 12.30:
        return '(Gentle Breeze) 7.38 < speed <= 12.30'
    elif 12.30 < x <= 17.67:
        return '(Moderate Breeze) 12.30 < speed <= 17.67'
    elif 17.67 < x <= 23.93:
        return '(Fresh Breeze) 17.67 < speed <= 23.93'
    elif 23.93 < x <= 30.87:
        return '(Strong Breeze) 23.93 < speed <= 30.87'
    elif 30.87 < x <= 38.25:
        return '(High Wind)30.87 < speed <= 38.25'
    elif 38.25 < x <= 46.30:
        return '(Gale) 38.25 < speed <= 46.30'
    elif 46.30 < x <= 54.58:
        return '(Strong Gale) 46.30 < speed <= 54.58'
    elif 54.58 < x <= 63.53:
        return '(Storm) 54.58 < speed <= 63.53'
    elif 63.53 < x <= 72.92:
        return '(Violent Storm) 63.53 < speed <= 72.92' 
    else:
        return '(Hurricane) speed > 72.92'

def discretize_rainfall(x):
    if math.isnan(x):
        return 'Unknown'
    elif x <= 0.1:
        return '(Light) rate <= 0.1'
    elif 0.1 < x <= 0.39:
        return '(Moderate) 0.1 < rate <= 0.3'
    elif 0.39 < x <= 2.0:
        return '(Heavy) 0.39 < rate <= 2.0'
    else:
        return '(Violent) rate > 2.0'

us_acidents['Temperature(F)'] = us_acidents["Temperature(F)"].map(discretize_temp)
us_acidents['Humidity(%)'] = us_acidents['Humidity(%)'].map(discretize_humidity)
us_acidents['Visibility(mi)'] = us_acidents['Visibility(mi)'].map(discretize_visibility)
us_acidents['Wind_Speed(mph)'] = us_acidents['Wind_Speed(mph)'].map(discretize_windspeed)
us_acidents['Precipitation(in)'] = us_acidents['Precipitation(in)'].map(discretize_rainfall)

us_acidents.fillna({'Weather_Condition':'Unknown', 'Wind_Direction':'Unknown'}, inplace=True)

In [260]:
us_acidents[['Temperature(F)', 'Humidity(%)', 'Precipitation(in)', 'Visibility(mi)', 'Weather_Condition', 'Wind_Direction']].isna().sum()

Temperature(F)       0
Humidity(%)          0
Precipitation(in)    0
Visibility(mi)       0
Weather_Condition    0
Wind_Direction       0
dtype: int64

In [261]:
weather_dimension = us_acidents[['WeatherKey', 'Temperature(F)', 'Visibility(mi)', 'Humidity(%)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)','Weather_Condition']]

### LocalTime Dimension


In [262]:
us_acidents['LocalTimeKey'] = pd.RangeIndex(start=1, stop=us_acidents.shape[0]+1)

In [263]:
us_acidents['StartTime'] = pd.DatetimeIndex(us_acidents['Start_Datetime']).time
us_acidents['EndTime'] = pd.DatetimeIndex(us_acidents['End_Datetime']).time

In [264]:
us_acidents

,ID,TMC,Severity,DateKey,StartDay,StartMonth,StartYear,EndDay,EndMonth,EndYear,...,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,LocalTimeKey,StartTime,EndTime
0,A-1,201,3,1,8,February,2016,8,February,2016,...,False,False,False,Night,Night,Night,Night,1,05:46:00,11:00:00
1,A-2,201,2,2,8,February,2016,8,February,2016,...,False,False,False,Night,Night,Night,Day,2,06:07:59,06:37:59
2,A-3,201,2,3,8,February,2016,8,February,2016,...,False,True,False,Night,Night,Day,Day,3,06:49:27,07:19:27
3,A-4,201,3,4,8,February,2016,8,February,2016,...,False,False,False,Night,Day,Day,Day,4,07:23:34,07:53:34
4,A-5,201,2,5,8,February,2016,8,February,2016,...,False,True,False,Day,Day,Day,Day,5,07:39:07,08:09:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496838,A-510722,201,2,496839,26,July,2017,26,July,2017,...,False,False,False,Day,Day,Day,Day,496839,17:51:40,18:21:32
496839,A-510723,201,2,496840,26,July,2017,26,July,2017,...,False,False,False,Day,Day,Day,Day,496840,17:51:54,18:21:47
496840,A-510724,201,2,496841,26,July,2017,26,July,2017,...,False,False,False,Day,Day,Day,Day,496841,17:56:24,18:26:15
496841,A-510725,201,2,496842,26,July,2017,26,July,2017,...,False,False,False,Day,Day,Day,Day,496842,18:04:35,18:34:10


In [265]:
def define_period_of_day(start_hour, civil_tw, naut_tw, ast_tw, sunrise_sunset) :
    if (civil_tw != naut_tw) or (naut_tw != ast_tw) :
        return "Dusk"
    elif dt.time(8, 0, 0) < start_hour < dt.time(10, 0, 0) :
        return 'Morning Rush Hour'
    elif dt.time(12, 30, 0) < start_hour < dt.time(14, 30, 0) :
        return 'Lunch Time'
    elif dt.time(16, 0, 0) < start_hour < dt.time(18, 0, 0) :
        return 'Evening Rush Hour'
    else:
        return sunrise_sunset

us_acidents['PeriodOfDay'] = us_acidents.apply(lambda x: define_period_of_day(x['StartTime'], x['Civil_Twilight'], x['Nautical_Twilight'], x['Astronomical_Twilight'], x['Sunrise_Sunset']), axis=1)

In [266]:
localtime_dimension = us_acidents[['LocalTimeKey', 'StartTime', 'PeriodOfDay', 'Timezone', 'EndTime']]

### StandardTime Dimension

In [267]:
us_acidents["Timezone"].value_counts()

US/Eastern     204775
US/Pacific     186641
US/Central     105395
US/Mountain        32
Name: Timezone, dtype: int64

In [268]:
# Creating the Standard Time Key
us_acidents['StandardTimeKey'] = pd.RangeIndex(start=1, stop=us_acidents.shape[0]+1)

In [269]:
import pytz

def convert_timezone(timestamp, old_timezone, new_timezone) :
    localized_timestamp = pytz.timezone(old_timezone).localize(timestamp)
    new_timezone_timestamp = localized_timestamp.astimezone(pytz.timezone(new_timezone))
    return dt.time(new_timezone_timestamp.hour, new_timezone_timestamp.minute, new_timezone_timestamp.second)

us_acidents['StandardStartTime'] = us_acidents.apply(lambda x: convert_timezone(x['Start_Datetime'], x['Timezone'], "Etc/GMT"), axis=1)
us_acidents['StandardEndTime'] = us_acidents.apply(lambda x: convert_timezone(x['End_Datetime'], x['Timezone'], "Etc/GMT"), axis=1)

In [270]:
us_acidents["Standard_Timezone"] = "GMT"
us_acidents[['StandardTimeKey','StartTime','EndTime','Timezone','StandardStartTime','StandardEndTime', 'Standard_Timezone']]
standardtime_dimension = us_acidents[['StandardTimeKey', 'StandardStartTime','StandardEndTime', 'Standard_Timezone']]

In [271]:
standardtime_dimension

,StandardTimeKey,StandardStartTime,StandardEndTime,Standard_Timezone
0,1,10:46:00,16:00:00,GMT
1,2,11:07:59,11:37:59,GMT
2,3,11:49:27,12:19:27,GMT
3,4,12:23:34,12:53:34,GMT
4,5,12:39:07,13:09:07,GMT
...,...,...,...,...
496838,496839,22:51:40,23:21:32,GMT
496839,496840,22:51:54,23:21:47,GMT
496840,496841,22:56:24,23:26:15,GMT
496841,496842,23:04:35,23:34:10,GMT


In [272]:
us_acidents.rename(columns={"Severity": "SeverityKey"}, inplace=True)

In [273]:
def correspond_severity(x):
    if x == 1 :
        return "Low impact - Short delay in traffic."
    elif x == 2:
        return "Moderate impact - Medium delay in traffic."
    elif x == 3:
        return "High impact - Considerable delay in traffic. Low risk of casualties."
    else:
        return "Significative impact - Long delay in traffic. High risk of casualties."
    
us_acidents["SeverityDescription"] = us_acidents["SeverityKey"].map(correspond_severity)

severity_dimension = us_acidents[["SeverityKey", "SeverityDescription"]]

severity_dimension.drop_duplicates(subset=['SeverityKey', 'SeverityDescription'], keep='last', inplace=True)
severity_dimension.sort_values(by='SeverityKey', inplace=True)
severity_dimension.reset_index(inplace=True)
severity_dimension.drop(columns=['index'], inplace=True)

<ipython-input-273-35d6df668891>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  severity_dimension.drop_duplicates(subset=['SeverityKey', 'SeverityDescription'], keep='last', inplace=True)
<ipython-input-273-35d6df668891>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  severity_dimension.sort_values(by='SeverityKey', inplace=True)
C:\Users\darsh\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

### Facts Table

In [277]:
us_acidents['Duration']=(us_acidents['End_Datetime']-us_acidents['Start_Datetime'])

In [278]:
facts_table = us_acidents[['ID', 'LocalTimeKey','StandardTimeKey', 'DateKey','LocationKey','WeatherKey','POI_Key','SeverityKey','TMC','Duration','Distance(mi)']]

In [ ]:
date_dimension.head(5)

#### Dimension LocalTime

In [284]:
standardtime_dimension.head(5)

,StandardTimeKey,StandardStartTime,StandardEndTime,Standard_Timezone
0,1,10:46:00,16:00:00,GMT
1,2,11:07:59,11:37:59,GMT
2,3,11:49:27,12:19:27,GMT
3,4,12:23:34,12:53:34,GMT
4,5,12:39:07,13:09:07,GMT


In [287]:
location_dimension.head(5)

,LocationKey,Street,City,County,State,Subregion,Region,AlcoholConsumptionRate,NumberOfVehicleRegistrations,UrbanRuralClassification,MaximumSpeedLimit
0,1,I-70 E,Dayton,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80
1,2,Brice Rd,Reynoldsburg,Franklin,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Medium Metro,(Medium) 70 <= limit < 80
2,3,State Route 32,Williamsburg,Clermont,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Large Fringe Metro,(Medium) 70 <= limit < 80
3,4,I-75 S,Dayton,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80
4,5,Miamisburg Centerville Rd,Dayton,Montgomery,Ohio,East North Central,Midwest,(Medium) 2.0 < rate <= 2.5,(High) 10000000 < registrations <= 20000000,Small Metro,(Medium) 70 <= limit < 80


#### Dimension Weather

In [289]:
weather_dimension.head(5)

,WeatherKey,Temperature(F),Visibility(mi),Humidity(%),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
0,1,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,Calm,Unknown,(Light) rate <= 0.1,Light Rain
1,2,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,Calm,Unknown,(Light) rate <= 0.1,Light Rain
2,3,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,SW,(Light Breeze) 3.36 < speed <= 7.38,(Light) rate <= 0.1,Overcast
3,4,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,SW,(Light Breeze) 3.36 < speed <= 7.38,(Light) rate <= 0.1,Mostly Cloudy
4,5,(Low) 32.0 < temp <= 50.0,(Good) 5.0 < visibility <= 20.0,(Very High) rate > 75.0,SW,(Light Breeze) 3.36 < speed <= 7.38,(Light) rate <= 0.1,Mostly Cloudy


#### Dimension 'POI'

In [291]:
poi_dimension.head(5)

,POI_Key,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,1,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
1,2,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
2,3,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present
3,4,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present
4,5,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Not Present,Present,Not Present


#### Dimension 'TMC'

In [293]:
event_code.head(5)

,TMC_Key,Event
0,1,traffic problem
1,2,queuing traffic (with average speeds Q). Dange...
2,11,overheight warning system triggered
3,12,"(Q) accident(s), traffic being directed around..."
4,16,"closed, rescue and recovery work in progress"


#### Dimension 'Severity'

In [295]:
severity_dimension

,SeverityKey,SeverityDescription
0,1,Low impact - Short delay in traffic.
1,2,Moderate impact - Medium delay in traffic.
2,3,High impact - Considerable delay in traffic. L...
3,4,Significative impact - Long delay in traffic. ...


### Fact Table

In [297]:
facts_table.head(5)

,ID,LocalTimeKey,StandardTimeKey,DateKey,LocationKey,WeatherKey,POI_Key,SeverityKey,TMC,Duration,Distance(mi)
0,A-1,1,1,1,1,1,1,3,201,0 days 05:14:00,0.01
1,A-2,2,2,2,2,2,2,2,201,0 days 00:30:00,0.01
2,A-3,3,3,3,3,3,3,2,201,0 days 00:30:00,0.01
3,A-4,4,4,4,4,4,4,3,201,0 days 00:30:00,0.01
4,A-5,5,5,5,5,5,5,2,201,0 days 00:30:00,0.01


In [299]:
import psycopg2 as pg

#### Creating the tables

In [300]:
create_statements_list = []
drop_statements_list = []

create_local_time_table = ('CREATE TABLE local_time_dimension', """create table local_time_dimension(
                                local_time_key int,
                                start_time time,
                                period_of_day char(32), 
                                timezone char(16),
                                end_time time,
                                primary key (local_time_key)
                            )""")
create_statements_list.append(create_local_time_table)

create_standard_time_table = ('CREATE TABLE standard_time_dimension', """create table standard_time_dimension(
                                    standard_time_key int,
                                    start_time time,
                                    standard_timezone char(16),
                                    end_time time,
                                    primary key (standard_time_key)
                            )""")
create_statements_list.append(create_standard_time_table)

create_location_table = ('CREATE TABLE location_dimension', """create table location_dimension(
                                location_key int, 	
                                street char(64), 	
                                city char(64), 	
                                county char(64),	
                                state char(64),	
                                subregion char(32), 
                                region char(32), 	
                                alcohol_consumption_rate char(32),	
                                number_of_vehicle_registrations char(64),	
                                urban_rural_classification char(32),	
                                maximum_speed_limit char(32),
                                primary key (location_key)
                            )""")

create_statements_list.append(create_location_table)
                            
create_date_table = ('CREATE TABLE date_dimension', """create table date_dimension(
                                    date_key int,
                                    start_day int,
                                    day_of_week char(16),
                                    weekday_or_weekend char(8),
                                    week_number int,
                                    work_day char(16),
                                    holiday char(64),
                                    school_break char(16),
                                    start_month char(16),
                                    quarter int,
                                    start_year int,
                                    end_day int,
                                    end_month char(16),
                                    end_year int,
                                    primary key (date_key)
                            )""")
create_statements_list.append(create_date_table)

create_poi_table = ('CREATE TABLE poi_dimension', """create table poi_dimension(
                                    poi_key int,
                                    presence_of_amenity char(16),
                                    presence_of_bump char(16),
                                    presence_of_crossing char(16),
                                    presence_of_giveway char(16),
                                    presence_of_junction char(16),
                                    presence_of_noexit char(16),
                                    presence_of_rail_way char(16),
                                    presence_of_roundabout char(16),
                                    presence_of_station char(16),
                                    presence_of_stop char(16),
                                    presence_of_traffic_calming char(16),
                                    presence_of_traffic_signal char(16),
                                    presence_of_turning_loop char(16),
                                    primary key (poi_key)
                            )""")
create_statements_list.append(create_poi_table)
                            
create_tmc_table = ('CREATE TABLE tmc_dimension', """create table tmc_dimension(
                            tmc_key int,
                            event char(256),
                            primary key (tmc_key)  
                    )
                    """)
create_statements_list.append(create_tmc_table)

create_weather_table = ('CREATE TABLE weather_dimension', """create table weather_dimension(
                                weather_key int,
                                temperature char(128),
                                visibility char(128),
                                humidity char(128),
                                wind_direction char(128),
                                wind_speed char(128),
                                precipitation char(128),
                                weather_condition char(128),
                                primary key (weather_key)
                   )
                   """)
create_statements_list.append(create_weather_table)

create_severity_table = ('CREATE TABLE severity_dimension', """create table severity_dimension(
                                severity_key int,
                                description char(128),
                                primary key (severity_key)
                   )
                   """)
create_statements_list.append(create_severity_table)

create_facts_table = ('CREATE TABLE facts_table', """create table facts_table(
                                accidentid varchar(32),
                                local_timekey int,
                                standard_timekey int,
                                datekey int,
                                locationkey int,
                                weatherkey int,
                                poikey int,
                                severitykey int,
                                tmckey int,
                                duration interval,
                                distance float,
                                primary key (accidentid),
                                foreign key (local_timekey) references local_time_dimension (local_time_key),
                                foreign key (standard_timekey) references standard_time_dimension (standard_time_key),
                                foreign key (datekey) references date_dimension (date_key),
                                foreign key (locationkey) references location_dimension (location_key),
                                foreign key (weatherkey) references weather_dimension (weather_key),
                                foreign key (poikey) references poi_dimension (poi_key),
                                foreign key (severitykey) references severity_dimension (severity_key),
                                foreign key (tmckey) references tmc_dimension (tmc_key)
                )
                """)
create_statements_list.append(create_facts_table)

drop_facts_table = ('DROP TABLE facts_table', "DROP TABLE IF EXISTS facts_table CASCADE")
drop_statements_list.append(drop_facts_table)

drop_local_time_table = ('DROP TABLE local_time_dimension', "DROP TABLE IF EXISTS local_time_dimension")
drop_statements_list.append(drop_local_time_table)
drop_standard_time_table = ('DROP TABLE standard_time_dimension', "DROP TABLE IF EXISTS standard_time_dimension")
drop_statements_list.append(drop_standard_time_table)
drop_location_table = ('DROP TABLE location_dimension', "DROP TABLE IF EXISTS location_dimension")
drop_statements_list.append(drop_location_table)
drop_date_table = ('DROP TABLE date_dimension', "DROP TABLE IF EXISTS date_dimension")
drop_statements_list.append(drop_date_table)
drop_poi_table = ('DROP TABLE poi_dimension', "DROP TABLE IF EXISTS poi_dimension")
drop_statements_list.append(drop_poi_table)
drop_weather_table = ('DROP TABLE weather_dimension', "DROP TABLE IF EXISTS weather_dimension")
drop_statements_list.append(drop_weather_table)
drop_severity_table = ('DROP TABLE severity_dimension', "DROP TABLE IF EXISTS severity_dimension")
drop_statements_list.append(drop_severity_table)
drop_tmc_table = ('DROP TABLE tmc_dimension', "DROP TABLE IF EXISTS tmc_dimension")
drop_statements_list.append(drop_tmc_table)

In [301]:
HOST = "127.0.0.1"
DATABASE = "accident_data_warehouse"
USER = "postgres"
PASSWORD = "1234" 

def execute_multiple_statements(statements):
    conn = pg.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    cur = conn.cursor()
    print('Executing multiple statements:')
    for statement, sql in statements:
        print('\t%s' % statement)
        cur.execute(sql)
    print('\nFinished!')
    cur.close()
    conn.commit()
    conn.close()

def execute_single_statement(statement):
    conn = pg.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    cur = conn.cursor()
    print('Executing %s ...' % statement[0])
    cur.execute(statement[1])
    print('\nFinished!')
    cur.close()
    conn.commit()
    conn.close()

def build_database():
    return execute_multiple_statements(create_statements_list)

def drop_database():
    return execute_multiple_statements(drop_statements_list)

drop_database()
build_database()

Executing multiple statements:
	DROP TABLE facts_table
	DROP TABLE local_time_dimension
	DROP TABLE standard_time_dimension
	DROP TABLE location_dimension
	DROP TABLE date_dimension
	DROP TABLE poi_dimension
	DROP TABLE weather_dimension
	DROP TABLE severity_dimension
	DROP TABLE tmc_dimension

Finished!
Executing multiple statements:
	CREATE TABLE local_time_dimension
	CREATE TABLE standard_time_dimension
	CREATE TABLE location_dimension
	CREATE TABLE date_dimension
	CREATE TABLE poi_dimension
	CREATE TABLE tmc_dimension
	CREATE TABLE weather_dimension
	CREATE TABLE severity_dimension
	CREATE TABLE facts_table

Finished!


#### Adding Data to the tables


#### Dimension 'Date'

In [302]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Date Dimension...")
for row in list(date_dimension.itertuples(index=False, name=None)):
    datekey,std,stm,sty,endd,endm,endy,dow,wkwkend,qtr,wknum,holi,workd,schbrk = row
    curs.execute("insert into date_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
        (datekey,std,stm,sty,endd,endm,endy,dow,wkwkend,qtr,wknum,holi,workd,schbrk))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Date Dimension...
Success!


Populating 'LocalTime' Dimension

In [303]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating LocalTime Dimension...")
for row in list(localtime_dimension.itertuples(index=False, name=None)):
    localkey,st,prd,tz,end = row
    curs.execute("insert into local_time_dimension values(%s, %s, %s, %s, %s)", (localkey,st,prd,tz,end))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating LocalTime Dimension...
Success!


#### Populating 'StandardTime'

In [304]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating StandardTime Dimension...")
for row in list(standardtime_dimension.itertuples(index=False, name=None)):
    standardkey, standardst, standardend, standardtz = row
    curs.execute("insert into standard_time_dimension values(%s, %s, %s, %s)", 
    (standardkey, standardst, standardtz, standardend))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating StandardTime Dimension...
Success!


#### Populating Location Dimension

In [305]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Location Dimension...")
for row in list(location_dimension.itertuples(index=False, name=None)):
    locationkey,street,city,county,state,reg,subreg,alc,veh,urc,spd = row
    curs.execute("insert into location_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
        (locationkey,street,city,county,state,reg,subreg,alc,veh,urc,spd))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Location Dimension...
Success!


#### Populating Severity Dimension

In [306]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Severity Dimension...")
for row in list(severity_dimension.itertuples(index=False, name=None)):
    sevkey,event = row
    curs.execute("insert into severity_dimension values(%s, %s)", (sevkey, event))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Severity Dimension...
Success!


#### Populating Weather Dimension

In [307]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Weather Dimension...")
for row in list(weather_dimension.itertuples(index=False, name=None)):
    weatherkey,temp,vis,hum,windir,winsp,prec,cond = row
    curs.execute("insert into weather_dimension values(%s, %s, %s, %s, %s, %s, %s, %s)", 
    (weatherkey,temp,vis,hum,windir,winsp,prec,cond))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Weather Dimension...
Success!


#### Populating POI Dimension

In [308]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating POI Dimension...")
for row in list(poi_dimension.itertuples(index=False, name=None)):
    poikey,ame,bump,cross,givw,junc,nox,rw,rnd,st,stop,calm,sig,loop = row
    curs.execute("insert into poi_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
    (poikey,ame,bump,cross,givw,junc,nox,rw,rnd,st,stop,calm,sig,loop))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating POI Dimension...
Success!


#### Populating TMC dimension

In [309]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating TMC Dimension...")
for row in list(event_code.itertuples(index=False, name=None)):
    tmckey, event = row
    curs.execute("insert into tmc_dimension values(%s, %s)", (tmckey, event))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating TMC Dimension...
Success!


#### Populating Facts Table

In [310]:
conn = pg.connect(host=HOST,database=DATABASE, user=USER, password=PASSWORD)
curs = conn.cursor()
print("Populating Facts' Table...")
for row in list(facts_table.itertuples(index=False, name=None)):
    id,ltk,stk,dtk,lck,wtrk,poik,svrk,tmck,dur,dist = row
    curs.execute("insert into facts_table values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", 
    (id,ltk,stk,dtk,lck,wtrk,poik,svrk,tmck,dur,dist))
curs.close()
conn.commit()
conn.close()
print("Success!")

Populating Facts' Table...
Success!
